# Experiment on the encoding and imputing technics for the Nam network, with and without Z ! 

In [10]:
run init.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done.


**`NAN` Second set of experiments deciding wheether `using indicator variable is useful`**

In [11]:
"""  Parameters of the experiment """
approach = 'ebm'

range_dataset_name = ['circles', 'moons', 'blobs']
range_ratio_of_missing_values = [0., 0.1, 0.2, 0.3]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.5, 0.25, 0.1]
range_missingness_pattern = [1, 3, 4, 5]

range_missing_data_handling = ['encoding'] #TODO THIS IS WHERE WE SHOULD DO IMUTING BASED ON THE  OTHER EXPERIMENT
imputation_method = 'without'
range_use_missing_indicator_variables = [True, False]

In [12]:
df = create_df(folder_names=['experiments_29_june_duke'])
df = df.append(pd.read_csv("/home01/sam/df_all_experiments.csv"))
df

,dataset_name,experiment_number,approach,missing_data_handling,imputation_method,use_missing_indicator_variables,num_samples,imbalance_ratio,missingness_pattern,missingness_mechanism,ratio_of_missing_values,missing_X1,missing_X2,missing_first_quarter,ratio_missing_per_class_0,ratio_missing_per_class_1,auc,Accuracy,F1,MCC,Sensitivity,Specificity,Precision,PPV,NPV,FNR,FDR,FOR,resolution,bandwidth,estimation_time_0,estimation_time_1
0,circles,188,nam,encoding,without,False,5000,0.5,3,MAR,0.3,True,False,True,NaN,NaN,0.9469,0.8653,0.8711,0.7394,0.9346,0.8000,0.8159,0.8159,0.9282,0.0654,0.1841,0.0718,20,0.2,NaN,NaN
1,circles,156,ebm,encoding,without,True,5000,0.1,3,MAR,0.2,True,False,True,NaN,NaN,NaN,0.9590,0.7450,0.7310,0.6590,0.9890,0.8570,0.8570,0.9670,0.3410,0.1430,0.0330,20,0.2,NaN,NaN
2,circles,186,nam,encoding,without,False,5000,0.5,3,MAR,0.1,True,False,True,NaN,NaN,0.9762,0.9117,0.9146,0.8260,0.9492,0.8745,0.8831,0.8831,0.9455,0.0508,0.1169,0.0545,20,0.2,NaN,NaN
3,circles,150,ebm,encoding,without,True,5000,0.1,2,MAR,0.0,True,True,True,NaN,NaN,NaN,0.9770,0.8830,0.8740,0.8210,0.9960,0.9560,0.9560,0.9790,0.1790,0.0440,0.0210,20,0.2,NaN,NaN
4,circles,212,nam,encoding,without,False,5000,0.1,1,MCAR,0.3,True,False,None,NaN,NaN,0.9387,0.9024,0.0000,NaN,0.0000,1.0000,NaN,NaN,0.9024,1.0000,NaN,0.0976,20,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,blobs,13,xgboost,encoding,without,True,5000,0.5,2,MAR,0.0,True,True,True,NaN,NaN,NaN,0.9790,0.9790,0.9580,0.9800,0.9780,0.9780,0.9780,0.9800,0.0200,0.0220,0.0200,20,0.2,NaN,NaN
863,blobs,5,xgboost,without,without,True,5000,0.5,4,MNAR,NaN,True,True,False,0.0,0.25,NaN,0.9840,0.9840,0.9680,0.9780,0.9900,0.9900,0.9900,0.9780,0.0220,0.0100,0.0220,20,0.2,NaN,NaN
864,blobs,14,xgboost,encoding,without,True,5000,0.5,2,MAR,0.1,True,True,True,NaN,NaN,NaN,0.9380,0.9370,0.8760,0.9350,0.9410,0.9390,0.9390,0.9380,0.0650,0.0610,0.0620,20,0.2,NaN,NaN
865,blobs,22,xgboost,encoding,without,True,5000,0.5,4,MNAR,NaN,True,True,False,0.1,0.30,NaN,0.9460,0.9470,0.8930,0.9680,0.9240,0.9270,0.9270,0.9670,0.0320,0.0730,0.0330,20,0.2,NaN,NaN


In [7]:
if False:
    df = create_df(folder_names=['experiments_24_june_duke', 'experiments_24_june', 'experiments_solidays', 'experiement_28_june', 'experiments_28_june_duke', 'experiments_28_june_exp_1'])
else:
    df = pd.read_csv("/home01/sam/notes_on_stats/doc/src/data/merged_df_29_june.csv")

In [14]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):
                        


                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          approach=approach,
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class_0=ratio_missing_per_class[0] if ratio_missing_per_class is not None else None):


                            print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                            print("Not doing this.")
                            continue
                            
                            
                            
                        print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                        missing_data_handling,
                                                                                                                                                          imputation_method, 
                                                                                                                                                          imbalance_ratio,
                                                                                                                                                          missingness_pattern,
                                                                                                                                                          ratio_of_missing_values, 
                                                                                                                                                              ratio_missing_per_class))



circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.0 ratio_per_class: None
circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.1 ratio_per_class: None
circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.2 ratio_per_class: None
circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.3 ratio_per_class: None
circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 2 ratio_missing:0.0 ratio_per_class: None
circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 2 ratio_missing:0.1 ratio_per_class: None
circles - imputation method: encoding missing_data_handling: without imbalance: 0.5 pattern: 2 ratio_missing:0.2 ratio_per_class: None
circles - imputation method: encoding missing_data_hand

In [15]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):




                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          approach=approach,
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class_0=ratio_missing_per_class[0] if ratio_missing_per_class is not None else None):


                            print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                            print("Not doing this.")
                            continue
                            
                            
                            
                        print("{} - {} - missing_data_handling {} imputation_method: {} use_missing_indicator_variables: {} imbalance_ratio: {} missingness_pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                                                                                    approach,
                                                                                                                                                                                                                    missing_data_handling,
                                                                                                                                                                                                                    imputation_method, 
                                                                                                                                                                                                                    use_missing_indicator_variables,
                                                                                                                                                                                                                    imbalance_ratio,
                                                                                                                                                                                                                    missingness_pattern,
                                                                                                                                                                                                                    ratio_of_missing_values, 
                                                                                                                                                                                                                    ratio_missing_per_class))

                        try:
                            start_time = time()

                            dataset = DatasetGenerator(dataset_name=dataset_name, 
                                                    num_samples=num_samples, 
                                                    imbalance_ratio=imbalance_ratio, 
                                                    missing_data_handling=missing_data_handling,
                                                    imputation_method=imputation_method,
                                                    verbosity=verbosity)

                            # Creation of the missingness
                            dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                                 ratio_of_missing_values=ratio_of_missing_values, 
                                                                 ratio_missing_per_class=ratio_missing_per_class)

                            dataset.split_test_train()

                            # Create the experiments
                            exp = Experiments(dataset_name, 
                                              dataset=dataset, 
                                              purpose='classification', 
                                              approach = approach, 
                                              use_missing_indicator_variables=use_missing_indicator_variables,
                                              previous_experiment=None, 
                                              save_experiment=save, 
                                              verbosity=verbosity, 
                                              debug=False, 
                                              proportion_train=PROPORTION_TRAIN, 
                                              resolution=RESOLUTION, 
                                              bandwidth=BANDWIDTH,
                                              random_state=RANDOM_STATE)


                            # Estimate the distributions 
                            exp.fit()

                            # Estimate the distributions 
                            exp.predict()

                            # Plot results
                            #exp.plot()


                            end_time = time()

                            hours, rest = divmod(end_time - start_time, 3600)
                            minutes, seconds = divmod(rest, 60)
                            print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                        except:
                            print("/!\. Error!\n")
                            print("{} - {} - missing_data_handling {} imputation_method: {} use_missing_indicator_variables: {} imbalance_ratio: {} missingness_pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                                                                                    approach,
                                                                                                                                                                                                                    missing_data_handling,
                                                                                                                                                                                                                    imputation_method, 
                                                                                                                                                                                                                    use_missing_indicator_variables,
                                                                                                                                                                                                                    imbalance_ratio,
                                                                                                                                                                                                                    missingness_pattern,
                                                                                                                                                                                                                    ratio_of_missing_values, 
                                                                                                                                                                                                                    ratio_missing_per_class)) 


circles - ebm - missing_data_handling encoding imputation_method: without use_missing_indicator_variables: True imbalance_ratio: 0.5 missingness_pattern: 1 ratio_missing:0.0 ratio_per_class: None
Doing experiment 216!
Done (0h 0m and 3.35s).

circles - ebm - missing_data_handling encoding imputation_method: without use_missing_indicator_variables: True imbalance_ratio: 0.5 missingness_pattern: 1 ratio_missing:0.1 ratio_per_class: None
Doing experiment 217!
Done (0h 0m and 2.24s).

circles - ebm - missing_data_handling encoding imputation_method: without use_missing_indicator_variables: True imbalance_ratio: 0.5 missingness_pattern: 1 ratio_missing:0.2 ratio_per_class: None
Doing experiment 218!
Done (0h 0m and 2.19s).

circles - ebm - missing_data_handling encoding imputation_method: without use_missing_indicator_variables: True imbalance_ratio: 0.5 missingness_pattern: 1 ratio_missing:0.3 ratio_per_class: None
Doing experiment 219!
Done (0h 0m and 2.84s).

circles - ebm - missing_data